In [1]:
from analysis.get_dataframe_from_results import get_pandas_dataset
from analysis.dataset_graphs import plot_graphs_of_dataset_loc
from analysis.get_tables_results import create_table_cleaned
from analysis.get_results import bar_chart_fix_position_cleaned
from analysis.get_results import sucess_vs_position_cleaned,get_latex_table_with_verif_stats
import utils.global_variables as gl
import llm.llm_configurations as llm_configurations
import llm.llm_pipeline as llm_pipeline
from llm.llm_create import create_llm

import utils.global_variables as gl
import pandas as pd
import shutil

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [2]:
RESULT_DIR = gl.BASE_PATH / "results/dafny_llm_results_cost_statistics"
shutil.rmtree(RESULT_DIR, ignore_errors=True)
RESULT_DIR.mkdir(parents=True, exist_ok=True)
DATASET_DIR = gl.DAFNY_ASSERTION_DATASET

base_prompt = gl.BASE_PROMPT
localization_base_prompt = gl.LOCALIZATION_BASE_PROMPT
system_prompt = gl.SYSTEM_PROMPT
# Test point for the test dataset and for the llm_results_dir_Test
global_options = llm_pipeline.GlobalOptions(
    dafny_exec= gl.DAFNY_EXEC,
    temp_dir= gl.TEMP_FOLDER,
    llm_results_dir=RESULT_DIR,
    assertion_dataset_path=DATASET_DIR
)

run_options = llm_pipeline.RunOptions(
    number_assertions_to_test=-1, # if -1 test all assetions
    number_rounds=1, # number of indepedent rounds in each assertion
    number_retries_chain=1, #number of retries it tries to ask fix for previous wrong andwer
    add_error_message = True,
    remove_empty_lines=True,
    change_assertion_per_text="", #Assertion with oracle to retrieve position
    base_prompt=base_prompt,
    localization_base_prompt=localization_base_prompt,
    examples_to_augment_prompt_type = llm_pipeline.ExampleStrategies.NONE, # "RANDOM", "DYNAMIC", "NONE", "EMBEDDED" "TFIDF" all options    
                                               # DYNAMIC represents my code/error message embeddings
    examples_weight_of_error_message = 0.5, # Wieght of the erro message in case of DYNAMIC in relantion with code                                 
    number_examples_to_add=3,
    # Fields ignore for all except LLM_EXAMPLE
    examples_to_augment_prompt_type_pos= llm_pipeline.ExampleStrategies.NONE, # "RANDOM", "DYNAMIC", "NONE", "EMBEDDED" "TFIDF" all options    
                                               # DYNAMIC represents my code/error message embeddings
    examples_weight_of_error_message_pos = 0.5, # Wieght of the erro message in case of DYNAMIC in relantion with code                                 
    number_examples_to_add_pos=3,

    limit_example_length_bytes = 1200, # Option Not implemented
    verifier_output_filter_warnings = True, # If 1 only errors are passed in the verifier ouput string
    system_prompt=system_prompt,
    skip_original_verification= True,
    localization = llm_pipeline.LocStrategies.ORACLE,
    # Options related with if will prompt LLM or gathered from saved results
    skip_verification= True,
    only_verify = False,
    only_get_location = False,
    only_get_assert_candidate = False,
)

run_options.examples_to_augment_prompt_type=llm_pipeline.ExampleStrategies.DYNAMIC # "RANDOM", "DYNAMIC", "NONE", "EMBEDDED" "TFIDF" all options     
run_options.examples_weight_of_error_message = 0.25 # Wieght of the erro message in case of DYNAMIC in relantion with code                                 
run_options.number_examples_to_add=3


In [3]:
llm_cost_stub = create_llm("cost","cost_stub_almost_real")

print("Pipeline")
print("Laurel Better")
run_options.localization = llm_pipeline.LocStrategies.LAUREL_BETTER
llm_pipeline.evaluate_all(llm_cost_stub, global_options, run_options)
llm_cost_stub.get_cost_statistics(llm_configurations.MODEL_REGISTRY["claude-haiku-4.5"])
llm_cost_stub.reset_all_measurement()

print("LLM")
run_options.localization = llm_pipeline.LocStrategies.LLM
llm_pipeline.evaluate_all(llm_cost_stub, global_options, run_options)
llm_cost_stub.get_cost_statistics(llm_configurations.MODEL_REGISTRY["claude-haiku-4.5"])
llm_cost_stub.reset_all_measurement()

print("LLM_EXAMPLE")
run_options.examples_to_augment_prompt_type_pos = llm_pipeline.ExampleStrategies.DYNAMIC
run_options.examples_weight_of_error_message_pos = 0.25
run_options.number_examples_to_add_pos = 3
run_options.localization = llm_pipeline.LocStrategies.LLM_EXAMPLE
llm_pipeline.evaluate_all(llm_cost_stub, global_options, run_options)
llm_cost_stub.get_cost_statistics(llm_configurations.MODEL_REGISTRY["claude-haiku-4.5"])
llm_cost_stub.reset_all_measurement()

print("HYBRID")
run_options.localization = llm_pipeline.LocStrategies.HYBRID
llm_pipeline.evaluate_all(llm_cost_stub, global_options, run_options)
llm_cost_stub.get_cost_statistics(llm_configurations.MODEL_REGISTRY["claude-haiku-4.5"])
llm_cost_stub.reset_all_measurement()


Pipeline
Laurel Better
LEN assertion groups 448


Eval Groups: cost (Active Cores:6) (seq):   0%|          | 0/448 [00:00<?, ?it/s]/home/ricostynha/.local/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ricostynha/.cache/huggingface/modules/transformers_modules/jinaai/jina_hyphen_bert_hyphen_v2_hyphen_qk_hyphen_post_hyphen_norm/3baf9e3ac750e76e8edd3019170176884695fb94/configuration_bert.py:29: UserWarning: optimum is not installed. To use OnnxConfig and BertOnnxConfig, make sure that `optimum` package is installed
  warnings.warn("optimum is not installed. To use OnnxConfig and BertOnnxConfig, make sure that `optimum` package is installed")
Eval Groups: cost (Active Cores:6) (seq): 100%|██████████| 448/448 [00:24<00:00, 18.13it/s]


Expected Prices Model name: cost Model id: us.anthropic.claude-haiku-4-5-20251001-v1:0
Statistic                               Value               
Total Prompts                           448                 
Total Chars Prompted                    3202623             
Total Chars Response                    323456              
Total Tokens Input                      1067541.00          
Total Tokens Output                     107818.67           
Total Tokens Output Reason              0.00                
Cost Input ($)                          1.067541            
Cost Output ($)                         0.539093            
Cost Output Reason($)                   0.000000            
Total Cost ($)                          1.606634            
LLM
LEN assertion groups 448


Eval Groups: cost (Active Cores:6) (seq): 100%|██████████| 448/448 [00:16<00:00, 27.73it/s]


Expected Prices Model name: cost Model id: us.anthropic.claude-haiku-4-5-20251001-v1:0
Statistic                               Value               
Total Prompts                           896                 
Total Chars Prompted                    4724112             
Total Chars Response                    327040              
Total Tokens Input                      1574704.00          
Total Tokens Output                     109013.33           
Total Tokens Output Reason              0.00                
Cost Input ($)                          1.574704            
Cost Output ($)                         0.545067            
Cost Output Reason($)                   0.000000            
Total Cost ($)                          2.119771            
LLM_EXAMPLE
LEN assertion groups 448


Eval Groups: cost (Active Cores:6) (seq): 100%|██████████| 448/448 [00:31<00:00, 14.20it/s]


Expected Prices Model name: cost Model id: us.anthropic.claude-haiku-4-5-20251001-v1:0
Statistic                               Value               
Total Prompts                           896                 
Total Chars Prompted                    6716655             
Total Chars Response                    327040              
Total Tokens Input                      2238885.00          
Total Tokens Output                     109013.33           
Total Tokens Output Reason              0.00                
Cost Input ($)                          2.238885            
Cost Output ($)                         0.545067            
Cost Output Reason($)                   0.000000            
Total Cost ($)                          2.783952            
HYBRID
LEN assertion groups 448


Eval Groups: cost (Active Cores:6) (seq): 100%|██████████| 448/448 [00:31<00:00, 14.04it/s]

Expected Prices Model name: cost Model id: us.anthropic.claude-haiku-4-5-20251001-v1:0
Statistic                               Value               
Total Prompts                           896                 
Total Chars Prompted                    6739519             
Total Chars Response                    327040              
Total Tokens Input                      2246506.33          
Total Tokens Output                     109013.33           
Total Tokens Output Reason              0.00                
Cost Input ($)                          2.246506            
Cost Output ($)                         0.545067            
Cost Output Reason($)                   0.000000            
Total Cost ($)                          2.791573            


In [ ]:
llm_cost_stub = create_llm("cost","cost_stub_response_dafnybench")
print("Method that would answer the complete program on the output")
run_options.localization = llm_pipeline.LocStrategies.ORACLE

# compare also with examples on the prompt
run_options.examples_to_augment_prompt_type=llm_pipeline.ExampleStrategies.DYNAMIC # "RANDOM", "DYNAMIC", "NONE", "EMBEDDED" "TFIDF" all options     
run_options.examples_weight_of_error_message = 0.25 # Wieght of the erro message in case of DYNAMIC in relantion with code                                 
run_options.number_examples_to_add=3 

llm_pipeline.evaluate_all(llm_cost_stub, global_options, run_options)
llm_cost_stub.get_cost_statistics(llm_configurations.MODEL_REGISTRY["claude-haiku-4.5"])
llm_cost_stub.reset_all_measurement()
# Can be tested by only performing the inference prompt, but assuming that llm returns a response with full prompt

Method that wouls andser the complete program on output
LEN assertion groups 448


Eval Groups: cost (Active Cores:6) (seq): 100%|██████████| 448/448 [00:16<00:00, 26.75it/s]

Expected Prices Model name: cost Model id: us.anthropic.claude-haiku-4-5-20251001-v1:0
Statistic                               Value               
Total Prompts                           448                 
Total Chars Prompted                    3280573             
Total Chars Response                    554985              
Total Tokens Input                      1093524.33          
Total Tokens Output                     184995.00           
Total Tokens Output Reason              0.00                
Cost Input ($)                          1.093524            
Cost Output ($)                         0.924975            
Cost Output Reason($)                   0.000000            
Total Cost ($)                          2.018499            
